# Base Models 

In [24]:
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
nlp = English()
import gensim.downloader

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import warnings
warnings.filterwarnings('default')

In [2]:
df1 = pd.read_csv('cd.csv')
df1.head(2)

,id,tweet,label,domain
0,2.263108e+16,@FC_HELP can I return online purchases to a Ho...,0,apparel
1,2.336408e+16,@FC_Help Hi - I'm writing a piece for MSN Him ...,0,apparel


In [3]:
glv = gensim.downloader.load("glove-twitter-200") #https://radimrehurek.com/gensim/downloader.html?

In [27]:
df2 = df1.copy()
p = 0
n = 0
l = 0
m = 0

for count, tweet in enumerate(df2.tweet):
    dummy = []
    m += len(nlp(tweet))
    if len(nlp(tweet)) > l:
        l = len(nlp(tweet)) 
    for word in nlp(tweet):
        try:
            dummy.append(glv.wv[str(word)])
            p+=1
        except:
            rand = np.zeros(200)
            dummy.append(rand)
            n+=1
            
    df2.at[count, 'tweet'] = dummy

C:\Users\ADMIN\anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [28]:
print('Words in Glove dictionary :', p)
print('Words not in Glove dictionary :', n)
print('Longest tweet length :', l)
print('Average tweet length :', m/len(df1))

Words in Glove dictionary : 45418
Words not in Glove dictionary : 15066
Longest tweet length : 60
Average tweet length : 17.53667729776747


In [29]:
max_len = 20
df3 = df2.copy()

for count, tweet in enumerate(df2.tweet):
    tw_len = len(tweet)
    #print(tw_len)
    #print(tweet)
    content = df2.tweet[count]
    if tw_len < max_len:
        for i in range(max_len - tw_len):
            content.append(np.zeros((1, 200))[0])
        df2.at[count, 'tweet'] = np.stack(content)
    else:
        df2.at[count, 'tweet'] = np.stack(df2.at[count, 'tweet'][:max_len]) 
        
    df2.at[count, 'tweet'] = np.ndarray.flatten(df2.tweet[count]).reshape((1, max_len * 200)).astype(np.float32)  
    df3.at[count, 'tweet'] = np.array(df2.tweet[count]).reshape((1, max_len, 200)).astype(np.float32)

## Base Model 1

In [17]:
trainx1 = np.stack(df2.tweet).reshape((len(df2), 1, max_len * 200))
trainy1 = np.stack(df2.label).reshape((len(df2), 1))

train_data1 = tf.data.Dataset.from_tensor_slices((trainx1, trainy1))

In [18]:
base_model_1 = keras.Sequential(
    [
        layers.Dense(100, activation = "relu", name = "layer1"),
        layers.Dense(1, activation = "sigmoid", name = "layer2"),
    ])

# Call model on a test input
x = df2.tweet[1]
y = base_model_1.predict(x)

In [19]:
base_model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)               multiple                  400100    
_________________________________________________________________
layer2 (Dense)               multiple                  101       
Total params: 400,201
Trainable params: 400,201
Non-trainable params: 0
_________________________________________________________________


In [20]:
base_model_1.compile(optimizer = tf.keras.optimizers.Adam(), loss = tf.keras.losses.BinaryCrossentropy(), 
                     metrics = tf.keras.metrics.Recall(name = 'recall'))
base_model_1.fit(train_data1, epochs = 5)

Epoch 1/5
3449/3449 [==============================] - 11s 3ms/step - loss: 0.5199 - recall: 0.6526
Epoch 2/5
3449/3449 [==============================] - 12s 3ms/step - loss: 0.2620 - recall: 0.8369
Epoch 3/5
3449/3449 [==============================] - 11s 3ms/step - loss: 0.0904 - recall: 0.9481
Epoch 4/5
3449/3449 [==============================] - 12s 3ms/step - loss: 0.0625 - recall: 0.9643
Epoch 5/5
3449/3449 [==============================] - 12s 3ms/step - loss: 0.0483 - recall: 0.9748


In [25]:
comp = '@adidas I ordered a sneakers last month and yet to recieve. Will I get it atleat by this month?'

test = []

for word in nlp(comp):
    try:
        test.append(glv.wv[str(word)])
        p+=1
    except:
        rand = np.zeros(200)
        test.append(rand)
        n+=1
        
for i in range(max_len - len(test)):
    test.append(np.zeros((1, 200))[0])
if len(test) > max_len:
    test = test[:max_len]

test = np.stack(test)
test = np.ndarray.flatten(test)

print(float(base_model_1(test.reshape((1, max_len * 200)))[0][0]) * 100, '% the tweet is a complaint.')

90.49748182296753 % the tweet is a complaint.


C:\Users\ADMIN\anaconda3\envs\python37\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
C:\Users\ADMIN\anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


##### Uncomment the following cell for training using cross validation

In [18]:
# kfold = KFold(n_splits = 3, shuffle = True)

# fold_no = 1
# for train, test in kfold.split(trainx, trainy):

#     # Define the model architecture
#     base_model_1 = keras.Sequential(
#         [
#             layers.Dense(100, activation = "relu", name = "layer1"),
#             layers.Dense(1, activation = "sigmoid", name = "layer2"),
#         ])

#     # Compile the model
#     base_model_1.compile(optimizer = tf.keras.optimizer.Adam(learning_rate = 0.01, name = 'Adam'), 
#                          loss = tf.keras.losses.BinaryCrossentropy(), metrics = tf.keras.metrics.Recall(name = 'recall'))

#     # Fit data to model
#     history = base_model_1.fit(trainx[train], trainy[train], epochs = 20, verbose = 0)

#     # Generate generalization metrics
#     scores = base_model_1.evaluate(trainx[test], trainy[test], verbose=0)
#     print(f'Score fold number {fold_no}: {base_model_1.metrics_names[0]} of {scores[0]}; {base_model_1.metrics_names[1]} of {scores[1]*100}%')

#     fold_no = fold_no + 1

Score fold number 1: loss of 1.1272565126419067; recall of 58.91088843345642%
Score fold number 2: loss of 1.2913845777511597; recall of 58.23389291763306%
Score fold number 3: loss of 1.147955060005188; recall of 61.369192600250244%


## Base Model 2

In [43]:
trainx2 = np.stack(df3.tweet).reshape((len(df2), 1, max_len, 200))
trainy2 = np.stack(df3.label).reshape((len(df2), 1))

train_data2 = tf.data.Dataset.from_tensor_slices((trainx2, trainy2))

In [44]:
base_model_2 = keras.Sequential(
    [
        keras.layers.LSTM(units = 50),
        keras.layers.Dense(1, activation = 'sigmoid')
    ])

x = df3.tweet[1]
y = base_model_2.predict(x)

print(base_model_2.summary())

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               multiple                  50200     
_________________________________________________________________
dense_4 (Dense)              multiple                  51        
Total params: 50,251
Trainable params: 50,251
Non-trainable params: 0
_________________________________________________________________
None


In [53]:
base_model_2.compile(optimizer = tf.keras.optimizers.Adam(), loss = tf.keras.losses.BinaryCrossentropy(), 
                     metrics = tf.keras.metrics.Recall(name = 'recall'))
base_model_2.fit(train_data2, epochs = 5)

Epoch 1/5
3449/3449 [==============================] - 20s 6ms/step - loss: 0.1208 - recall: 0.9351
Epoch 2/5
3449/3449 [==============================] - 21s 6ms/step - loss: 0.0916 - recall: 0.9578
Epoch 3/5
3449/3449 [==============================] - 22s 6ms/step - loss: 0.0922 - recall: 0.9537
Epoch 4/5
3449/3449 [==============================] - 21s 6ms/step - loss: 0.0601 - recall: 0.9716
Epoch 5/5
3449/3449 [==============================] - 18s 5ms/step - loss: 0.0591 - recall: 0.9708


In [54]:
comp = '@adidas I ordered a sneakers last month and yet to recieve. Will I get it atleat by this month?'

test = []

for word in nlp(comp):
    try:
        test.append(glv.wv[str(word)])
        p+=1
    except:
        rand = np.zeros(200)
        test.append(rand)
        n+=1
        
for i in range(max_len - len(test)):
    test.append(np.zeros((1, 200))[0])
if len(test) > max_len:
    test = test[:max_len]

test = np.stack(test)
test = test.reshape((1, max_len, 200))
print(float(base_model_2(test)[0][0]) * 100, '% the tweet is a complaint.')

88.11834454536438 % the tweet is a complaint.


C:\Users\ADMIN\anaconda3\envs\python37\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys
